In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

import torch
import random

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import convert_wmdp_data_to_prompt, convert_list_of_dicts_to_dict_of_lists
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis, ActivationStoreAnalysis
from unlearning.metrics import modify_and_calculate_metrics, calculate_metrics_list, create_df_from_metrics
from unlearning.feature_attribution import calculate_cache

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np
import pandas as pd
import itertools
from transformer_lens import utils

from jaxtyping import Float
from torch import Tensor

import einops

from pathlib import Path

import plotly.express as px
from unlearning.var import REPO_ID, SAE_MAPPING
import pickle

from unlearning.metrics import all_permutations

from unlearning.metrics import calculate_metrics_side_effects
from unlearning.feature_attribution import find_topk_features_given_prompt, test_topk_features


In [3]:
# Load main SAE for gemma-2b-it
filename = hf_hub_download(repo_id=REPO_ID, filename=SAE_MAPPING['gemma_2b_it_resid_pre_9'])
sae = load_saved_sae(filename)
model = model_store_from_sae(sae)

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


...

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda


In [4]:
# pass in the dataset as an argument so no need to load in everytime
dataset = load_dataset("cais/wmdp", "wmdp-bio", split='test')

answers = [x['answer'] for x in dataset]
questions = [x['question'] for x in dataset]
choices_list = [x['choices'] for x in dataset]

prompts = [convert_wmdp_data_to_prompt(question, choices, prompt_format=None)
           for question, choices in zip(questions, choices_list)]


Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

In [21]:
# Hyper parameters for automation process
question_ids_correct = np.genfromtxt("../data/question_ids/gemma-2b-it/all/wmdp-bio_correct.csv", dtype=int)
questions_ids_correct_train = np.genfromtxt("../data/question_ids/gemma-2b-it/train/wmdp-bio_correct.csv", dtype=int)
topk_per_prompt = 20

unlearning_dataset = ['wmdp-bio']
side_effect_dataset_names =  ['high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']
all_dataset_names = ['loss_added', 'wmdp-bio', 'high_school_us_history', 'college_computer_science', 'high_school_geography', 'human_aging', 'college_biology']


## First get the TopK features by attribution per prompt and find the features that modify the probability

In [9]:
feature_per_prompt = {}

known_good_features = []

question_ids = questions_ids_correct_train

for j, question_id in enumerate(question_ids):

    question_id = int(question_id)
    print(f"Question ID: {question_id}, {j + 1}/{len(question_ids)}")
    
    prompt = prompts[question_id]
    choices = choices_list[question_id]
    answer = answers[question_id]
    question = questions[question_id]

    topk_features_unique, feature_attributions, topk_features, all_feature_activations, logit_diff_grad, topk_feature_attributions = find_topk_features_given_prompt(model,
                                                           prompt,
                                                           question,
                                                           choices,
                                                           answer,
                                                           sae,
                                                           hook_point=sae.cfg.hook_point)

    intervention_results, feature_ids_to_probs, good_features, partially_unlearned = test_topk_features(model,
                                                                                   sae,
                                                                                   question_id,
                                                                                   topk_features_unique[:topk_per_prompt],
                                                                                   known_good_features=known_good_features,
                                                                                   multiplier=30)
    

    feature_per_prompt[question_id] = good_features
    
    known_good_features = list(set([item for sublist in feature_per_prompt.values() for item in sublist]))

    

Question ID: 1147, 1/86


100%|██████████| 20/20 [00:18<00:00,  1.05it/s]


Question ID: 357, 2/86


100%|██████████| 18/18 [00:21<00:00,  1.19s/it]


Question ID: 800, 3/86


...

100%|██████████| 13/13 [00:18<00:00,  1.44s/it]


Question ID: 352, 85/86


100%|██████████| 14/14 [00:14<00:00,  1.04s/it]


Question ID: 841, 86/86


100%|██████████| 16/16 [00:14<00:00,  1.10it/s]


In [51]:
np.array(known_good_features).shape

(104,)

In [10]:
np.savetxt("known_good_features_gemma1_2b_train.csv", np.array(known_good_features))

## Calculate side-effects

In [15]:
known_good_features = np.genfromtxt("../eoin_notebooks/known_good_features_gemma1_2b_all.csv")

In [53]:
# Calculate metrics

main_ablate_params = {
                      'multiplier': 30,
                      'intervention_method': 'clamp_feature_activation',
                     }


sweep = {
         'features_to_ablate': np.array(known_good_features),
        }


dataset_names = all_dataset_names[2:-1]

metrics_list = calculate_metrics_side_effects(model,
                                              sae,
                                              main_ablate_params,
                                              sweep,
                                              dataset_names=dataset_names,
                                              thresh=1,
                                              target_metric='correct',
                                              split='all'
                                              )


100%|██████████| 104/104 [14:12<00:00,  8.20s/it]


In [54]:
feature_ids_zero_side_effect = [x['ablate_params']['features_to_ablate'] for x in metrics_list]
np.array(feature_ids_zero_side_effect).shape

(59,)

In [55]:
np.savetxt("feature_ids_1_side_effect_gemma1_2b_train.csv", np.array(feature_ids_zero_side_effect))

In [8]:
np.array(feature_ids_zero_side_effect)

array([12289.,  3599.,  1557.,    32.,  4654.,  4687.,  5205., 11358.,
       10355.,  6263., 13431., 13443.,  3728., 13980.,  6308.,  4777.,
        4271.,  9399., 13010.,   741.,  5861.,  4342.,  9473., 12044.,
        4886., 15642., 16175.,  7983., 10051.,   842.,   338.,  5996.,
       11122., 12663.,  1406.,  6531., 10632., 11147.,  8596.,  5525.,
        7076., 10176.,  9163.,  8660., 16341.,  7638., 14296., 14819.,
        7140.,  4071., 12782., 12273., 15858.])

In [18]:
np.array(feature_ids_zero_side_effect)

array([ 3599,  1557,    32,  4654,  4687,  5205, 11358, 13431, 13443,
       13980,  6308,  4777,  4271, 13010,  5861,  4342,  9473, 12044,
       15642, 16175,  7983, 10051,  5996,  1406, 11147,  8596,  5525,
        7076,  9163,  8660, 16341, 14296,  4071, 12273])

In [57]:
# np.savetxt("feature_ids_zero_side_effect_gemma1_2b_all.csv", np.array(feature_ids_zero_side_effect))

## Then sort by loss added

In [7]:
feature_ids_zero_side_effect = np.genfromtxt("feature_ids_5_side_effect_gemma1_2b_train.csv")

In [8]:
activation_store = ActivationStoreAnalysis(sae.cfg, model)

buffer
dataloader


In [56]:
# Calculate metrics

main_ablate_params = {
                      'multiplier': 20,
                      'intervention_method': 'clamp_feature_activation',
                     }


sweep = {
         'features_to_ablate': feature_ids_zero_side_effect,
        }

metric_params = {'wmdp-bio': 
                 {
                       # 'target_metric': 'correct',
                       'question_subset': question_ids_correct,
                       'permutations': None,
                       'verbose': False,
                   }
                 }

dataset_names = all_dataset_names[:2]

n_batch_loss_added = 10

metrics_list_zero_side_effect = calculate_metrics_list(model,
                                      sae,
                                      main_ablate_params,
                                      sweep,
                                      dataset_names=dataset_names,
                                      metric_params=metric_params,
                                      include_baseline_metrics=False,
                                      n_batch_loss_added=n_batch_loss_added,
                                      activation_store=activation_store,
                                      split='all',
                                      verbose=False)


100%|██████████| 59/59 [19:56<00:00, 20.28s/it]


In [57]:
df_zero_side_effect = create_df_from_metrics(metrics_list_zero_side_effect)
df_zero_side_effect

,loss_added,wmdp-bio,wmdp-bio_prob
0,-7.152557e-08,0.988372,0.994141
1,2.254200e-03,0.994186,0.995605
2,-7.152557e-08,0.924419,0.983398
3,-7.152557e-08,1.000000,0.995605
4,3.876686e-05,0.994186,0.995605
5,5.426645e-04,0.994186,0.994629
6,-7.152557e-08,0.912791,0.970703
7,1.673937e-04,1.000000,0.995117
8,-6.308937e-03,0.982558,0.992676
9,1.539111e-03,0.994186,0.995117


In [59]:
isorted = df_zero_side_effect.query("`wmdp-bio` < 1").sort_values("loss_added").index.values
feature_ids_zero_side_effect_sorted = np.array(feature_ids_zero_side_effect)[isorted]
feature_ids_zero_side_effect_sorted

array([ 6712., 13431.,  7983.,  4342.,  6308., 15642., 11358.,  8660.,
        8596.,  9163., 10051., 15858.,  7956., 12044.,  9473.,  5633.,
       13443.,  2523.,  5205., 12782.,  4654.,  1557., 12273., 13686.,
        3102.,  5895.,  8679.,  6531.,    32.,  5109., 15755.,  7076.,
       16175.,  5525.,  7803.,  4071.,  4687.,  9048., 11147.,  3599.,
        5001.,  6263.,   842.,  5861.,  7140.])

In [13]:
isorted = df_zero_side_effect.query("`wmdp-bio` < 1").sort_values("loss_added").index.values
feature_ids_zero_side_effect_sorted = np.array(feature_ids_zero_side_effect)[isorted]
feature_ids_zero_side_effect_sorted

array([10632., 12663., 10355., 13431.,  3728.,  7983.,  4342.,  6308.,
       15642., 12289., 11358.,  8660., 12044., 10051.,  9163., 12782.,
       12273.,  8596., 15858., 13443.,  1557.,  5205.,  9473.,  4654.,
       11122.,  6531.,    32.,  7076.,   338.,  5525.,  4071.,  4687.,
       11147.,  3599., 10176.,  4886.,  6263.,   741.,  9399.,   842.,
        5861.,  7140.])

In [25]:
len(feature_ids_zero_side_effect_sorted)

51

In [61]:
len(feature_ids_zero_side_effect_sorted)

45

In [60]:
# np.savetxt("feature_ids_side_effect_1_sorted_gemma1_2b_all.csv", np.array(feature_ids_zero_side_effect_sorted))

## Now progressively add features sorted by loss

In [45]:
feature_ids_zero_side_effect_sorted_0 = np.genfromtxt("../eoin_notebooks/feature_ids_zero_side_effect_sorted_gemma1_2b_all.csv")

In [46]:
len(feature_ids_zero_side_effect_sorted_0)

26

In [9]:
feature_ids_zero_side_effect_sorted = np.array([10632., 12663., 10355., 13431.,  3728.,  7983.,  4342.,  6308.,
       15642., 12289., 11358.,  8660., 12044., 10051.,  9163., 12782.,
       12273.,  8596., 15858., 13443.,  1557.,  5205.,  9473.,  4654.,
       11122.,  6531.,    32.,  7076.,   338.,  5525.,  4071.,  4687.,
       11147.,  3599., 10176.,  4886.,  6263.,   741.,  9399.,   842.,
        5861.,  7140.])

In [62]:
feature_ids_zero_side_effect_sorted_1 = np.genfromtxt("../eoin_notebooks/feature_ids_side_effect_1_sorted_gemma1_2b_all.csv")

In [66]:
# Calculate metrics

main_ablate_params = {
                      'multiplier': 30,
                      'intervention_method': 'clamp_feature_activation',
                     }


sweep = {
         # 'features_to_ablate': [feature_ids_zero_side_effect_sorted[:i+1] for i in range(1)],
        'features_to_ablate': [feature_ids_zero_side_effect_sorted_1[:i+1] for i in range(0, 45, 4)],
         'multiplier': [25],
        }

metric_params = {'wmdp-bio': 
                 {
                       'target_metric': 'correct',
                       'permutations': None,
                       'verbose': False,
                   }
                 }

dataset_names = all_dataset_names[:]

n_batch_loss_added = 50

metrics_list_best_sorted = calculate_metrics_list(model,
                                      sae,
                                      main_ablate_params,
                                      sweep,
                                      dataset_names=dataset_names,
                                      metric_params=metric_params,
                                      include_baseline_metrics=False,
                                      n_batch_loss_added=n_batch_loss_added,
                                      activation_store=activation_store,
                                      split='test')


100%|██████████| 12/12 [15:16<00:00, 76.35s/it]


In [28]:
df = create_df_from_metrics(metrics_list_best_sorted)
df

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,0.000831,0.360465,0.857143,1.0,1.000000,0.875,0.5,0.901367,0.934570,0.998535,0.964844,0.969727,0.913086
1,0.011930,0.279070,0.928571,1.0,0.933333,0.875,0.5,0.887695,0.921387,0.998047,0.974609,0.973633,0.959961


In [64]:
df = create_df_from_metrics(metrics_list_best_sorted)
df

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,0.001021,0.988372,1.0,1.0,1.0,1.0000,1.000,0.991699,0.988281,0.999023,0.998047,0.985352,0.995117
1,0.003441,0.941860,1.0,1.0,1.0,1.0000,1.000,0.977539,0.988281,0.999023,0.998047,0.978516,0.994629
2,0.004461,0.930233,1.0,1.0,1.0,1.0000,1.000,0.976074,0.988281,0.999023,0.998047,0.978516,0.994629
3,0.004695,0.755814,1.0,1.0,1.0,1.0000,0.750,0.955566,0.988281,0.999023,0.998047,0.994629,0.961914
4,0.004920,0.790698,1.0,1.0,1.0,1.0000,0.750,0.949707,0.988281,0.999023,0.998047,0.994629,0.961914
5,0.004980,0.686046,1.0,1.0,1.0,0.9375,0.750,0.933105,0.988281,0.999023,0.998047,0.989258,0.956055
6,0.005137,0.488372,1.0,1.0,1.0,0.9375,0.625,0.894531,0.988281,0.999023,0.998047,0.989258,0.933594
7,0.011964,0.465116,1.0,1.0,1.0,0.9375,0.625,0.916504,0.988281,0.999023,0.997559,0.988281,0.932129
8,0.014977,0.465116,1.0,1.0,1.0,0.9375,0.625,0.921387,0.988281,0.999023,0.997559,0.988281,0.932129
9,0.020841,0.453488,1.0,1.0,1.0,0.9375,0.625,0.924805,0.987793,0.999023,0.997559,0.988281,0.928711


In [43]:
df = create_df_from_metrics(metrics_list_best_sorted)
df

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,-0.001097,0.965116,1.000000,1.00,1.000000,1.0000,1.000,0.984863,0.975586,0.999023,0.997559,0.983887,0.994629
1,0.004474,0.593023,0.857143,1.00,0.933333,0.9375,1.000,0.959473,0.949707,0.998535,0.966309,0.967285,0.989258
2,0.007272,0.534884,0.857143,1.00,0.933333,0.9375,1.000,0.957031,0.949707,0.998535,0.966797,0.967285,0.990723
3,0.007735,0.383721,0.857143,1.00,0.933333,1.0000,0.875,0.967773,0.949707,0.998047,0.966797,0.983398,0.988770
4,0.007744,0.383721,0.857143,1.00,0.933333,1.0000,0.750,0.965820,0.949707,0.998047,0.966309,0.964844,0.973145
5,0.008027,0.360465,0.857143,1.00,0.933333,1.0000,0.750,0.955078,0.949707,0.998047,0.966309,0.964355,0.972168
6,0.008792,0.279070,0.857143,1.00,0.933333,0.8750,0.625,0.894043,0.949707,0.998047,0.966309,0.977539,0.960938
7,0.010069,0.279070,0.928571,1.00,0.933333,0.8750,0.500,0.889160,0.922363,0.998047,0.974609,0.973633,0.959961
8,0.023916,0.290698,0.928571,1.00,0.866667,0.8750,0.500,0.894531,0.921387,0.998047,0.941406,0.973145,0.959961
9,0.028189,0.255814,0.928571,0.75,0.866667,0.8125,0.500,0.908691,0.929199,0.990234,0.941895,0.979980,0.960449


In [ ]:
# with open("df_metrics_5_side_effect.pkl", "wb") as f:
#     pickle.dump(df, f)

In [67]:
# # with open("df/_metrics_5_side_effect.pkl", "rb") as f:
import pickle

with open("../data/unlearn_results/gemma-2b-it/sae/clamp_feature_activation_m25_side_effect_s1_increasing_loss_added", "wb") as f:
    pickle.dump(metrics_list_best_sorted, f)

In [67]:
df = create_df_from_metrics(metrics_list_best_sorted)
df

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,-0.001015,0.988372,1.0,1.0,1.0,1.0,1.0,0.995117,0.987793,0.998535,0.996582,0.994141,0.997559
1,-0.001767,0.965116,1.0,1.0,1.0,1.0,1.0,0.989258,0.987793,0.998535,0.996582,0.994141,0.997559
2,-0.002136,0.953488,1.0,1.0,1.0,1.0,1.0,0.989258,0.987793,0.998535,0.996582,0.994141,0.997559
3,-0.002155,0.930233,1.0,1.0,1.0,1.0,1.0,0.984863,0.987793,0.998535,0.996582,0.994141,0.997559
4,-0.002642,0.918605,1.0,1.0,1.0,1.0,1.0,0.985352,0.987793,0.998535,0.996582,0.994141,0.997559
5,-0.002007,0.906977,1.0,1.0,1.0,1.0,1.0,0.985352,0.987793,0.998535,0.996582,0.994141,0.997559
6,-0.002163,0.895349,1.0,1.0,1.0,1.0,1.0,0.990234,0.987793,0.998535,0.996582,0.993164,0.997559
7,-0.002163,0.895349,1.0,1.0,1.0,1.0,1.0,0.990234,0.987793,0.998535,0.996582,0.993164,0.997559
8,-0.002186,0.872093,1.0,1.0,1.0,1.0,1.0,0.983887,0.987793,0.998535,0.996582,0.992676,0.997559
9,-0.002398,0.837209,1.0,1.0,1.0,1.0,1.0,0.977539,0.987793,0.998535,0.996582,0.992676,0.997559


In [26]:
# metrics_list_best_sorted

In [24]:
df = create_df_from_metrics(metrics_list_best_sorted)
df

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,-0.000212,0.976744,1.0,1.0,1.0,1.0000,1.0,0.994141,0.987305,0.998535,0.996582,0.994141,0.997559
1,0.004665,0.767442,1.0,1.0,1.0,0.9375,1.0,0.970215,0.987305,0.998535,0.996582,0.990234,0.923340
2,0.004644,0.732558,1.0,1.0,1.0,0.9375,1.0,0.972168,0.987305,0.998535,0.996582,0.989746,0.907227
3,0.005248,0.732558,1.0,1.0,1.0,0.9375,1.0,0.967773,0.987305,0.998535,0.996582,0.989746,0.909668


In [8]:
feature_ids_zero_side_effect_sorted = np.genfromtxt("feature_ids_zero_side_effect_sorted_gemma1_2b_all.csv")
feature_ids_zero_side_effect = np.genfromtxt("feature_ids_zero_side_effect_gemma1_2b_all.csv")

In [27]:
# Calculate metrics

main_ablate_params = {
                      'intervention_method': 'clamp_feature_activation',
                      # 'jump': 0.5,
                     }


sweep = {
         'features_to_ablate': [feature_ids_zero_side_effect_sorted[:-3]],
         'multiplier': [20, 30],
         # 'jump': [0., 1],
        }

metric_params = {'wmdp-bio': 
                 {
                       'target_metric': 'correct',
                       'permutations': all_permutations,
                       'verbose': False,
                   }
                 }

dataset_names = all_dataset_names[:2]

n_batch_loss_added = 20

metrics_list_best_sorted4 = calculate_metrics_list(model,
                                      sae,
                                      main_ablate_params,
                                      sweep,
                                      dataset_names=dataset_names,
                                      metric_params=metric_params,
                                      include_baseline_metrics=False,
                                      n_batch_loss_added=n_batch_loss_added,
                                      activation_store=activation_store,
                                      split='all')


100%|██████████| 2/2 [05:22<00:00, 161.47s/it]


In [29]:
df4 = create_df_from_metrics(metrics_list_best_sorted4)
df4

,loss_added,wmdp-bio,wmdp-bio_prob
0,0.003645,0.690891,0.934082
1,0.027801,0.592539,0.915039


In [110]:
df3 = create_df_from_metrics(metrics_list_best_sorted3)
df3

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,0.004161,0.651163,1.0,1.0,1.0,1.0,0.866667,0.925293,0.994629,0.998535,0.99707,0.993164,0.961914


In [116]:
from unlearning.metrics import generate_ablate_params_list

In [118]:
ablate_params_list = generate_ablate_params_list(main_ablate_params, sweep)
for x in ablate_params_list:
    print(x['jump'], x['multiplier'])
    


0.0 20
0.5 20
1 20
1.5 20
0.0 40
0.5 40
1 40
1.5 40


In [81]:
df2 = create_df_from_metrics(metrics_list_best_sorted2)
df2

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,-0.003495,0.848837,1.0,1.0,1.0,1.0,1.000000,0.965332,0.994629,0.999023,0.99707,0.993164,0.984863
1,-0.001551,0.755814,1.0,1.0,1.0,1.0,0.866667,0.951172,0.994629,0.998535,0.99707,0.993652,0.952637
2,0.004161,0.651163,1.0,1.0,1.0,1.0,0.866667,0.925293,0.994629,0.998535,0.99707,0.993164,0.961914
3,0.014927,0.575581,1.0,1.0,1.0,1.0,0.866667,0.902344,0.994629,0.998535,0.99707,0.992676,0.984863
4,0.030548,0.523256,1.0,1.0,1.0,1.0,0.866667,0.898438,0.994141,0.998535,0.99707,0.992188,0.950195


In [71]:
df2 = create_df_from_metrics(metrics_list_best_sorted2)
df2

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,-0.0064,0.616279,1.0,1.0,1.0,1.0,1.0,0.921387,0.98877,0.998535,0.996582,0.993164,0.995117


In [73]:
df2 = create_df_from_metrics(metrics_list_best_sorted2)
df2

,loss_added,wmdp-bio,high_school_us_history,college_computer_science,high_school_geography,human_aging,college_biology,wmdp-bio_prob,high_school_us_history_prob,college_computer_science_prob,high_school_geography_prob,human_aging_prob,college_biology_prob
0,0.019185,0.651163,1.0,1.0,1.0,1.0,0.866667,0.923828,0.986816,0.998535,0.99707,0.992676,0.961914


In [104]:
metrics_list_best_sorted2[0]['human_aging']

{'mean_correct': 0.875,
 'total_correct': 14,
 'is_correct': array([1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       dtype=float32),
 'output_probs': array([[8.1062e-06, 1.1921e-07, 1.2517e-06, 9.9316e-01],
        [6.9336e-01, 1.4946e-02, 7.4646e-02, 2.1021e-01],
        [9.9854e-01, 4.9651e-05, 7.6771e-05, 4.4107e-06],
        [9.9902e-01, 9.5367e-07, 1.1921e-06, 7.3314e-06],
        [1.4853e-04, 5.8413e-06, 9.9463e-01, 4.3511e-06],
        [9.9805e-01, 1.1921e-07, 1.1921e-07, 2.3842e-07],
        [6.3777e-05, 1.3268e-04, 9.9707e-01, 3.8385e-05],
        [1.1325e-06, 1.4901e-06, 9.9707e-01, 8.6427e-06],
        [5.7817e-06, 1.7881e-07, 5.3644e-07, 9.9219e-01],
        [1.2927e-01, 8.4521e-01, 2.3438e-02, 7.5483e-04],
        [1.0700e-03, 9.9561e-01, 4.2915e-06, 1.1325e-06],
        [1.8597e-04, 7.3314e-06, 9.9854e-01, 2.1935e-05],
        [1.1325e-06, 5.9605e-08, 9.9707e-01, 1.1921e-07],
        [5.9605e-08, 9.9951e-01, 7.1526e-07, 1.7881e-07],
        [1.3912e

In [120]:
# Calculate metrics

main_ablate_params = {
                      'multiplier': 30,
                      'intervention_method': 'clamp_feature_activation',
                     }


sweep = {
         'features_to_ablate': feature_ids_zero_side_effect[:1],
        }

metric_params = {'wmdp-bio': 
                    {
                       'target_metric': 'correct',
                       'permutations': None,
                    }
                }

dataset_names = all_dataset_names[2:6]

n_batch_loss_added = 10

metrics_list = calculate_metrics_side_effects(model,
                                      sae,
                                      main_ablate_params,
                                      sweep,
                                      dataset_names=dataset_names,
                                      thresh=0,
                                      split='train')


  0%|          | 0/1 [00:00<?, ?it/s]

13 13
1.0 1.0
4 4
1.0 1.0
15 15
1.0 1.0


100%|██████████| 1/1 [00:07<00:00,  7.13s/it]

15 16
0.9375 1.0


In [122]:
metrics_list

[]

In [114]:
metrics_list[0]['human_aging']['mean_correct']

1.0

In [108]:
metrics_list[0]['human_aging']

IndexError: list index out of range